In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.dirname(os.getcwd()))

import tensorflow as tf
import keras_tuner as kt

from src.helper import create_and_prepare_dataset

In [ ]:
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU")
else:
    print("TensorFlow is using CPU")

In [ ]:
PROCESSED_DATA_DIR = "../data/processed"
TRAINING_DATA_DIR = "../data/split/train"
VALIDATION_DATA_DIR = "../data/split/val"
TEST_DATA_DIR = "../data/split/test"

BATCH_SIZE = 5

In [ ]:
training_dataset = create_and_prepare_dataset(TRAINING_DATA_DIR, BATCH_SIZE)
validation_dataset = create_and_prepare_dataset(VALIDATION_DATA_DIR, BATCH_SIZE)
test_dataset = create_and_prepare_dataset(TEST_DATA_DIR, BATCH_SIZE)

In [ ]:
class MyHyperModel(kt.HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))

        for i in range(hp.Int('num_layers', 2, 5)):
            model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=1024, step=32),
                                            activation='relu'))

        model.add(tf.keras.layers.Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

In [ ]:
hypermodel = MyHyperModel(input_shape=(63,), num_classes=7)

tuner = kt.Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=15,
    hyperband_iterations=2,
    directory='../hyperband',
    project_name='layers_and_learning_rate'
)

tuner.search_space_summary()

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

tuner.search(training_dataset, 
             epochs=10, 
             validation_data=validation_dataset,
             callbacks=[early_stopping])

tuner.results_summary()

In [ ]:
best_tuning_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hyperparameters.values)
print(best_tuning_model.summary())

val_loss, val_accuracy = best_tuning_model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")
print(f"Validation loss: {val_loss}")